In [ ]:
# uv run vllm serve Qwen/Qwen3-VL-8B-Instruct-FP8 \
#   --limit-mm-per-prompt.video 0 \
#   --async-scheduling \
#   --gpu-memory-utilization 0.6 \
#   --max-num-seqs 128 \
#   --max-model-len 2000

In [ ]:
import json 
from src.utils import evaluate_accuracy

samples = []
with open('ua_asr_hypotheses_500.jsonl', 'r') as json_file:
    json_list = list(json_file)

for json_str in json_list:
    result = json.loads(json_str)
    samples.append(result)



In [2]:
prompt = """You are a Ukrainian language expert. Choose the most accurate transcription based on:
- Grammatical correctness
- Natural word combinations
- Proper word boundaries (e.g., "по європейськи" not "поєвропейськи")

Options:
{options}

Reply with only the selected transcription number."""

In [3]:
from openai import OpenAI

client = OpenAI(
    api_key="EMPTY",
    base_url="http://localhost:8000/v1",
    timeout=3600
)

def inference_model(prompt, options):
    prompt = prompt.format(options=options)
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt
                }
            ]
        }
    ]

    response = client.chat.completions.create(
        model="Qwen/Qwen3-VL-8B-Instruct-FP8",
        messages=messages,
        max_tokens=10
    )
    return response.choices[0].message.content

In [ ]:
from tqdm import tqdm
selected_hypotheses = []
for sample in tqdm(samples):
    options = ""
    for idx, hypothesis in enumerate(sample['hypotheses']):
        options += f"{idx + 1}. {hypothesis}\n"
    
    response = inference_model(prompt, options)
    try:
        selected_index = int(response.strip()) - 1
    except ValueError:
        selected_index = 0  # Default to first option if parsing fails

    selected_hypotheses.append(sample['hypotheses'][selected_index])

accuracy = evaluate_accuracy(selected_hypotheses, samples)
print(f"Model Accuracy: {accuracy:.2f}%")

Model Accuracy: 0.70%
